# Tables vs. views

Views have been described as "virtual tables". It's true that views are similar to tables in certain aspects, but there are key differences. In this exercise, you will organize these differences and similarities.

![VIEW_CONTROL](/home/nero/Documents/Estudos/DataCamp/SQL/courses/database-design/view_control.png)

**Amazing! It looks like you got the hang of tables and views. These characteristics can help make decisions about what needs to be a table or view.**

In [ ]:
# exercise 01

"""
Viewing views

Because views are very useful, it's common to end up with many of them in your database. It's important to keep track of them so that database users know what is available to them.

The goal of this exercise is to get familiar with viewing views within a database and interpreting their purpose. This is a skill needed when writing database documentation or organizing views.
"""

# Instructions

"""

    Query the information schema to get views.
    Exclude system views in the results.
---
Question

What does view1 do?
Possible answers:
    
    Returns the content records with reviewids that have been viewed more than 4000 times.
    
    Returns the content records that have reviews of more than 4000 characters. {Answer}
    
    Returns the first 4000 records in content.
---
Question

What does view2 do?
Possible answers:
    
    Returns 10 random reviews published in 2017.
    
    Returns the top 10 lowest scored reviews published in 2017.
    
    Returns the top 10 highest scored reviews published in 2017. {Answer}
"""

# solution

-- Get all non-systems views
SELECT * FROM information_schema.views
WHERE table_schema NOT IN ('pg_catalog', 'information_schema');


#----------------------------------#



# Conclusion

"""
Awesome! Did view1 and view2 give you more ideas of how views can be used? Note that unlike this exercise, you should always give views descriptive names for views!
"""

### Query output

table_catalog |	table_schema |	table_name |	view_definition |	check_option |	is_updatable |	is_insertable_into |	is_trigger_updatable |	is_trigger_deletable |	is_trigger_insertable_into


dataarchpost |	public |	view1 | SELECT content.reviewid,
    content.content
   FROM content
  WHERE (length(content.content) > 4000); |	NONE |	YES |	YES |	NO |	NO |	NO


dataarchpost |	public |	view2 |	 SELECT reviews.reviewid,
    reviews.title,
    reviews.score
   FROM reviews
  WHERE (reviews.pub_year = 2017)
  ORDER BY reviews.score DESC
 LIMIT 10; | NONE |	NO |	NO |	NO

In [ ]:
# exercise 02

"""
Creating and querying a view

Have you ever found yourself running the same query over and over again? Maybe, you used to keep a text copy of the query in your desktop notes app, but that was all before you knew about views!

In these Pitchfork reviews, we're particularly interested in high-scoring reviews and if there's a common thread between the works that get high scores. In this exercise, you'll make a view to help with this analysis so that we don't have to type out the same query often to get these high-scoring reviews.
"""

# Instructions

"""

    Create a view called high_scores that holds reviews with scores above a 9.
---

    Count the number of records in high_scores that are self-released in the label field of the labels table.

"""

# solution

-- Create a view for reviews with a score above 9
CREATE VIEW high_scores AS
SELECT * FROM REVIEWS
WHERE score > 9;

-- Count the number of self-released works in high_scores
SELECT COUNT(*) FROM high_scores
INNER JOIN labels ON high_scores.reviewid = labels.reviewid
WHERE label = 'self-released';

#----------------------------------#

# Conclusion

"""
Nice, you've just created and queried a view! It wasn't too hard, right? Views are great because they're easy to set up and use immediately thereafter.
"""

In [ ]:
# exercise 03

"""
Creating a view from other views

Views can be created from queries that include other views. This is useful when you have a complex schema, potentially due to normalization, because it helps reduce the JOINS needed. The biggest concern is keeping track of dependencies, specifically how any modifying or dropping of a view may affect other views.

In the next few exercises, we'll continue using the Pitchfork reviews data. There are two views of interest in this exercise. top_15_2017 holds the top 15 highest scored reviews published in 2017 with columns reviewid,title, and score. artist_title returns a list of all reviewed titles and their respective artists with columns reviewid, title, and artist. From these views, we want to create a new view that gets the highest scoring artists of 2017.
"""

# Instructions

"""

    
    Create a view called top_artists_2017 with artist from artist_title.
    
    To only return the highest scoring artists of 2017, join the views top_15_2017 and artist_title on reviewid.
    
    Output top_artists_2017.
---
Question

Which is the DROP command that would drop both top_15_2017 and top_artists_2017?
Possible answers:
    
    DROP VIEW top_15_2017 CASCADE; {Answer}
    
    DROP VIEW top_15_2017 RESTRICT;
    
    DROP VIEW top_artists_2017 RESTRICT;
    
    DROP VIEW top_artists_2017 CASCADE;
"""

# solution

CREATE VIEW top_artists_2017 AS
SELECT artist_title.artist
FROM top_15_2017
INNER JOIN artist_title
ON top_15_2017.reviewid = artist_title.reviewid;

SELECT * FROM top_artists_2017;

#----------------------------------#

# Conclusion

"""
That's right! Because top_artists_2017 depends on top_15_2017, the CASCADE parameter indicates both should be dropped.
"""

In [ ]:
# exercise 04

"""
Granting and revoking access

Access control is a key aspect of database management. Not all database users have the same needs and goals, from analysts, clerks, data scientists, to data engineers. As a general rule of thumb, write access should never be the default and only be given when necessary.

In the case of our Pitchfork reviews, we don't want all database users to be able to write into the long_reviews view. Instead, the editor should be the only user able to edit this view.
"""

# Instructions

"""

    Revoke all database users' update and insert privileges on the long_reviews view.
    Grant the editor user update and insert privileges on the long_reviews view.

"""

# solution

-- Revoke everyone's update and insert privileges
REVOKE UPDATE, INSERT ON long_reviews FROM PUBLIC; 

-- Grant the editor update and insert privileges 
GRANT UPDATE, INSERT ON long_reviews TO editor; 

#----------------------------------#

# Conclusion

"""
Great! The editor user is now the only person who can UPDATE and INSERT on the long_reviews view.
"""

In [ ]:
# exercise 05

"""
Updatable views

In a previous exercise, we've used the information_schema.views to get all the views in a database. If you take a closer look at this table, you will notice a column that indicates whether the view is updatable.

Which views are updatable?
"""

# Instructions

"""
Possible answers:
    
    long_reviews and top_25_2017
    
    top_25_2017
    
    long_reviews {Answer}
    
    top_25_2017 and artist_title
"""

# solution

SELECT * FROM information_schema.views
WHERE is_updatable = 'YES';

#----------------------------------#

# Conclusion

"""
Nice! long_reviews is updatable because it's made from one table and doesn't have any special clauses.
"""

In [ ]:
# exercise 06

"""
Redefining a view

Unlike inserting and updating, redefining a view doesn't mean modifying the actual data a view holds. Rather, it means modifying the underlying query that makes the view. In the last video, we learned of two ways to redefine a view: (1) CREATE OR REPLACE and (2) DROP then CREATE. CREATE OR REPLACE can only be used under certain conditions.

The artist_title view needs to be appended to include a column for the label field from the labels table.
"""

# Instructions

"""
Question

Can the CREATE OR REPLACE statement be used to redefine the artist_title view?
Possible answers:
    
    Yes, as long as the label column comes at the end. {Answer}
    
    No, because the new query requires a JOIN with the labels table.
    
    No, because a new column that did not exist previously is being added to the view.
    
    Yes, as long as the label column has the same data type as the other columns in artist_title
---

    Use CREATE OR REPLACE to redefine the artist_title view.
    Respecting artist_title's original columns of reviewid, title, and artist, add a label column from the labels table.
    Join the labels table using the reviewid field.

"""

# solution

-- Redefine the artist_title view to have a label column
CREATE OR REPLACE VIEW artist_title AS
SELECT reviews.reviewid, reviews.title, artists.artist, labels.label
FROM reviews
INNER JOIN artists
ON artists.reviewid = reviews.reviewid
INNER JOIN labels
ON labels.reviewid = artists.reviewid;

SELECT * FROM artist_title;

#----------------------------------#

# Conclusion

"""
Terrific! You redefined the artist_title succesfully using the CREATE OR REPLACE statement. If we had wanted to change the order of the columns completely, we would have had to drop the table and then create a new one using the same name.
"""

# Materialized versus non-materialized

Materialized and non-materialized are two distinct categories of views. In this exercise, you will organize their differences and similarities.

![MATERIALIZED](/home/nero/Documents/Estudos/DataCamp/SQL/courses/database-design/materialized.png)

**Fantastic! It looks like you got the hang of their similarities and differences. These characteristics can help make decisions about when to use a materialized or non-materialized view.**

In [ ]:
# exercise 07

"""
Creating and refreshing a materialized view

The syntax for creating materialized and non-materialized views are quite similar because they are both defined by a query. One key difference is that we can refresh materialized views, while no such concept exists for non-materialized views. It's important to know how to refresh a materialized view, otherwise the view will remain a snapshot of the time the view was created.

In this exercise, you will create a materialized view from the table genres. A new record will then be inserted into genres. To make sure the view has the latest data, it will have to be refreshed.
"""

# Instructions

"""

    Create a materialized view called genre_count that holds the number of reviews for each genre.

    Refresh genre_count so that the view is up-to-date.

"""

# solution

-- Create a materialized view called genre_count 
CREATE MATERIALIZED VIEW genre_count AS
SELECT genre, COUNT(*) 
FROM genres
GROUP BY genre;

INSERT INTO genres
VALUES (50000, 'classical');

-- Refresh genre_count
REFRESH MATERIALIZED VIEW genre_count;

SELECT * FROM genre_count;

#----------------------------------#

# Conclusion

"""
Nice! If we didn't include that REFRESH statement, genre_count would not have a row for the classical genre since that genre did not exist before our INSERT statement.
"""

# Managing materialized views

Why do companies use pipeline schedulers, such as Airflow and Luigi, to manage materialized views?

### Possible Answers


    To set up a data warehouse and make sure tables have the most up-to-date data.
    
    
    To refresh materialized views with consideration to dependences between views. {Answer}
    
    
    To convert non-materialized views to materialized views.
    
    
    To prevent the creation of new materialized views when there are too many dependencies.

**Exactly! These pipeline schedulers help visualize dependencies and create a logical order for refreshing views.**